## Naive Bayesian Classifier

For each week, the feature set consists of mean return (μ) and volatility (σ).  

- Training Set: Boeing (BA) weekly stock data for years 2020–2022
- Testing Set: Boeing (BA) weekly stock data for years 2023–2024


In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

In [2]:
ba_volatility = pd.read_csv('../Inertia Trading/ba_weekly_return_volatility.csv')
ba_detailed = pd.read_csv("../Inertia Trading/ba_weekly_return_detailed.csv")

labels = ba_detailed[['Year','Week_Number', 'label']].drop_duplicates()

ba_volatility = ba_volatility.merge(
    labels, 
    on=['Year','Week_Number'], 
    how='inner'
)
ba_volatility.head()

,Year,Week_Number,mean_return,volatility,label
0,2020,0,-0.08400,0.118794,green
1,2020,1,-0.16120,1.584772,green
2,2020,2,-0.34560,1.269723,green
3,2020,3,-0.05525,2.818341,green
4,2020,4,-0.28880,1.510424,green


In [3]:
train_df = ba_volatility[ba_volatility['Year'] < 2023]
test_df = ba_volatility[ba_volatility['Year'] >= 2023]

In [4]:
features = ['mean_return', 'volatility']
X_train = train_df[features].values
le = LabelEncoder()
Y_train = le.fit_transform(train_df['label'].values)

X_test = test_df[features].values
Y_test = le.transform(test_df['label'].values)

NB_classifier = GaussianNB().fit(X_train, Y_train)
prediction = NB_classifier.predict(X_test)
acc = accuracy_score(Y_test, prediction)
print(f"Testing accuracy: {acc:.3f}")

Testing accuracy: 0.876


The model achieved 88% accuracy on the test set (Years 1-3). Compared to k-Nearest Neighbors(97%) and Logistic Regression(95%), the Naive Bayesian Classifier performed quite well. 

Also, as we observed before, our data is highly correlated and easily clustered visually, which is not an ideal scenario for Naive Bayes. However, it still managed to perform decently well.

### Testing Years Performance

In [5]:
cm = confusion_matrix(Y_test, prediction)
tn, fp, fn, tp = cm.ravel()
print(cm)

tpr = tp / (tp + fn)
tnr = tn / (fp + tn)
print(f"TPR: {tpr:.3f}")
print(f"TNR: {tnr:.3f}")

[[89  0]
 [13  3]]
TPR: 0.188
TNR: 1.000


Most of the testing weeks easily classified as "green" weeks, while the model struggled to identify "red" weeks. It is reasonable because of the imbalanced dataset.

### Buy-and-Hold vs Trading Strategy

In [6]:
test_df = test_df.copy()
test_df['predicted_label'] = le.inverse_transform(prediction)
test_df.head()

,Year,Week_Number,mean_return,volatility,label,predicted_label
157,2023,1,2.84125,1.618816,green,green
158,2023,2,0.12040,1.912990,green,green
159,2023,3,-0.87075,0.492591,green,green
160,2023,4,0.42620,0.875527,green,green
161,2023,5,-0.48200,1.689419,green,green


In [7]:
weekly_prices = (
    ba_detailed.groupby(['Year','Week_Number'], as_index=False)
    .agg(Open_w=('Open','first'), Close_w=('Close','last'))
    .sort_values(['Year','Week_Number'])
    .reset_index(drop=True)
)

weekly_prices_pred = weekly_prices.merge(test_df, on=['Year','Week_Number'], how='inner').sort_values(['Year','Week_Number']).reset_index(drop=True)
weekly_prices_pred.head()

,Year,Week_Number,Open_w,Close_w,mean_return,volatility,label,predicted_label
0,2023,1,192.949997,213.000000,2.84125,1.618816,green,green
1,2023,2,212.360001,214.130005,0.12040,1.912990,green,green
2,2023,3,212.710007,206.759995,-0.87075,0.492591,green,green
3,2023,4,208.360001,211.169998,0.42620,0.875527,green,green
4,2023,5,209.190002,206.009995,-0.48200,1.689419,green,green


In [8]:
# Buy-and-Hold
weekly_prices_pred_yr = weekly_prices_pred.groupby(["Year"]).agg(
    Close_y = ('Close_w', 'last')
    ).reset_index().sort_values(['Year']).reset_index(drop=True)


shares = (100 / weekly_prices_pred.iloc[0]['Open_w'])
portfolio=weekly_prices_pred_yr.copy()
portfolio["BuyHold"] = round(shares * weekly_prices_pred_yr['Close_y'], 2)

print(portfolio)

   Year     Close_y  BuyHold
0  2023  260.660004   135.09
1  2024  176.550003    91.50


In [9]:
# Trading Strategy
cash = 100
shares = 0
results = {}

weekly_prices_pred = weekly_prices_pred.sort_values(['Year','Week_Number']).reset_index(drop=True)

for i in range(len(weekly_prices_pred)):    
    week = weekly_prices_pred.iloc[i]
    next_week = weekly_prices_pred.iloc[i+1] if i+1 < len(weekly_prices_pred) else False
    
    if(shares == 0 and week['predicted_label'] == "green"):
        shares = cash / week['Open_w']
        cash = 0
    
    if (shares > 0 and (next_week['predicted_label'] == "red" or i==len(weekly_prices_pred)-1)):
        cash = shares * week['Close_w']
        shares = 0
        
    if week['Week_Number'] == 52: #store yearly earnings/losses
        yr_total = shares*week['Close_w'] if shares > 0 else cash
        results[week['Year']] = (yr_total)

portfolio["Gaussian"] = round(portfolio["Year"].map(results), 2)
print(portfolio)

   Year     Close_y  BuyHold  Gaussian
0  2023  260.660004   135.09    135.09
1  2024  176.550003    91.50    118.84


The trading strategy based on the Naive Bayes predictions outperformed the Buy-and-Hold strategy during the testing period. While for year-1 the results are similar, for year-2 we see Gaussian Naive Bayes strategy performed better ending with $118.84 compared to $91.50 from buy-and-hold.

However, overall portfolio results below that of kNN ($252.42) and Logistic Regression ($224.78) strategies, which is aligns with the model’s lower predictive accuracy.

## Naive Bayesian with custom density

In [10]:
from scipy import stats

In [11]:
train_df = ba_volatility[ba_volatility['Year'] < 2023]
test_df = ba_volatility[ba_volatility['Year'] >= 2023]

features = ['mean_return', 'volatility']
X_train = train_df[features].values
le = LabelEncoder()
Y_train = le.fit_transform(train_df['label'].values)
X_test = test_df[features].values
Y_test = le.transform(test_df['label'].values)

### Student-t Naive Bayesian Classifier
We model the likelihood of each feature using a student-t distribution, which is defined by 3 parameters:
- degrees of freedom df
- location parameter μ
- scale parameter s²

When degrees of freedom is large, Student-t distribution approaches normal. When df → 0, we get a distribution similar to normal but with fatter tails.

In [12]:
class StudentTNaiveBayes:
    def fit(self, X, y, df):
        self.classes = np.unique(y)
        self.mu = {}
        self.scale = {}
        self.prior_prob = {}
        self.df = df
        for c in self.classes:
            X_c = X[y == c]
            self.mu[c] = np.mean(X_c, axis=0)
            self.scale[c] = np.std(X_c, axis=0, ddof=1)
            self.prior_prob[c] = X_c.shape[0] / X.shape[0]
        return self
    
    def predict(self, X):
        log_probs = []
        for cls in self.classes:
            lp = np.sum(stats.t.logpdf(X, df=self.df, loc=self.mu[cls], scale=self.scale[cls]), axis=1) + np.log(self.prior_prob[cls])
            log_probs.append(lp)
        log_probs = np.array(log_probs).T
        predictions = np.argmax(log_probs, axis=1)
        return predictions

In [13]:
df = [0.5, 1, 5]
for df_value in df:
    Stud_t_classifier = StudentTNaiveBayes().fit(X_train, Y_train, df=df_value)
    prediction = Stud_t_classifier.predict(X_test)
    acc = accuracy_score(Y_test, prediction)
    print(f"Testing accuracy (df={df_value}): {acc:.3f}")
    
    cm = confusion_matrix(Y_test, prediction, labels=[0,1])
    print(cm)
    tn, fp, fn, tp = cm.ravel()
    tpr = tp/(tp+fn)
    tnr = tn/(fp+tn)
    print(f"TPR: {tpr:.3f}")
    print(f"TNR: {tnr:.3f}")

Testing accuracy (df=0.5): 0.848
[[89  0]
 [16  0]]
TPR: 0.000
TNR: 1.000
Testing accuracy (df=1): 0.848
[[89  0]
 [16  0]]
TPR: 0.000
TNR: 1.000
Testing accuracy (df=5): 0.848
[[89  0]
 [16  0]]
TPR: 0.000
TNR: 1.000


We see that changing the degrees of freedom does not impact the accuracy of the model on our dataset, meaning that the data distribution is close to normal. 

From the confusion matrix, we see that model struggles to identify red (cash) weeks, similar to Gaussian Naive Bayes. And most of the weeks are classified as green (buy) weeks which matches our imbalanced dataset.

In [14]:
# df=5
Stud_t_classifier = StudentTNaiveBayes().fit(X_train, Y_train, df=5)
prediction = Stud_t_classifier.predict(X_test)

test_df = test_df.copy()
test_df['predicted_label'] = le.inverse_transform(prediction)
test_df.head()

,Year,Week_Number,mean_return,volatility,label,predicted_label
157,2023,1,2.84125,1.618816,green,green
158,2023,2,0.12040,1.912990,green,green
159,2023,3,-0.87075,0.492591,green,green
160,2023,4,0.42620,0.875527,green,green
161,2023,5,-0.48200,1.689419,green,green


In [15]:
weekly_prices = (
    ba_detailed.groupby(['Year','Week_Number'], as_index=False)
    .agg(Open_w=('Open','first'), Close_w=('Close','last'))
    .sort_values(['Year','Week_Number'])
    .reset_index(drop=True)
)

weekly_prices_pred = weekly_prices.merge(test_df, on=['Year','Week_Number'], how='inner').sort_values(['Year','Week_Number']).reset_index(drop=True)
weekly_prices_pred.head()

,Year,Week_Number,Open_w,Close_w,mean_return,volatility,label,predicted_label
0,2023,1,192.949997,213.000000,2.84125,1.618816,green,green
1,2023,2,212.360001,214.130005,0.12040,1.912990,green,green
2,2023,3,212.710007,206.759995,-0.87075,0.492591,green,green
3,2023,4,208.360001,211.169998,0.42620,0.875527,green,green
4,2023,5,209.190002,206.009995,-0.48200,1.689419,green,green


In [16]:
# Buy-and-Hold
weekly_prices_pred_yr = weekly_prices_pred.groupby(["Year"]).agg(
    Close_y = ('Close_w', 'last')
    ).reset_index().sort_values(['Year']).reset_index(drop=True)


shares = (100 / weekly_prices_pred.iloc[0]['Open_w'])
portfolio=weekly_prices_pred_yr.copy()
portfolio["BuyHold"] = round(shares * weekly_prices_pred_yr['Close_y'], 2)

print(portfolio)

   Year     Close_y  BuyHold
0  2023  260.660004   135.09
1  2024  176.550003    91.50


In [17]:
# Trading Strategy
cash = 100
shares = 0
results = {}

weekly_prices_pred = weekly_prices_pred.sort_values(['Year','Week_Number']).reset_index(drop=True)

for i in range(len(weekly_prices_pred)):    
    week = weekly_prices_pred.iloc[i]
    next_week = weekly_prices_pred.iloc[i+1] if i+1 < len(weekly_prices_pred) else None
    
    if(shares == 0 and week['predicted_label'] == "green"):
        shares = cash / week['Open_w']
        cash = 0
    
    if (shares > 0 and ((next_week is None) or next_week['predicted_label'] == "red")):
        cash = shares * week['Close_w']
        shares = 0
        
    if week['Week_Number'] == 52: #store yearly earnings/losses
        yr_total = shares*week['Close_w'] if shares > 0 else cash
        results[week['Year']] = (yr_total)

portfolio["Stud-t.Dist"] = round(portfolio["Year"].map(results), 2)
print(portfolio)

   Year     Close_y  BuyHold  Stud-t.Dist
0  2023  260.660004   135.09       135.09
1  2024  176.550003    91.50        91.50


While the model maintained consistent accuracy, the trading strategy based on its predictions didn't outperfmorm the buy-and-hold. Overall, both methods resulted in the same final portfolio values for 2023 and 2024.

### Exponential Naive Bayesian Classifier
The Exponential Naive Bayes model assumes all features follow an exponential distribution and is defined by a single parameter λ (rate parameter).

In [18]:
train_df = ba_volatility[ba_volatility['Year'] < 2023]
test_df = ba_volatility[ba_volatility['Year'] >= 2023]

In [19]:
features = ['mean_return', 'volatility']
X_train = train_df[features].values
le = LabelEncoder()
Y_train = le.fit_transform(train_df['label'].values)
X_test = test_df[features].values
Y_test = le.transform(test_df['label'].values)

# need to make data positive for exponential distribution
shift = np.maximum(0.0, -X_train.min(axis=0))
X_train_pos = X_train + shift
X_test_pos  = X_test  + shift

In [20]:
class ExponentialNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.lambdas = {}
        self.prior_prob = {}
        for cls in self.classes:
            X_c = X[y == cls]
            self.lambdas[cls] = 1 / np.mean(X_c, axis=0)
            self.prior_prob[cls] = len(X_c) / len(X)
        return self
    
    def predict(self, X):
        log_probs = []    
        for cls in self.classes:
            lp = np.sum(stats.expon.logpdf(X, scale=1/self.lambdas[cls]), axis=1) + np.log(self.prior_prob[cls])
            log_probs.append(lp)
        log_probs = np.array(log_probs).T
        predictions = np.argmax(log_probs, axis=1)
        return predictions

In [21]:
ENB_classifier = ExponentialNaiveBayes().fit(X_train, Y_train)
prediction = ENB_classifier.predict(X_test)
acc = accuracy_score(Y_test, prediction)
print(f"Testing accuracy: {acc:.3f}")
cm = confusion_matrix(Y_test, prediction)
tn, fp, fn, tp = cm.ravel()
print(cm)
tpr = tp / (tp + fn)
tnr = tn / (fp + tn)
print(f"TPR: {tpr:.3f}")
print(f"TNR: {tnr:.3f}")

Testing accuracy: 0.152
[[ 0 89]
 [ 0 16]]
TPR: 1.000
TNR: 0.000


The model classified all weeks as class 1 (red), resulting in poor performance (15%).

In [22]:
test_df = test_df.copy()
test_df['predicted_label'] = le.inverse_transform(prediction)
test_df.head()

,Year,Week_Number,mean_return,volatility,label,predicted_label
157,2023,1,2.84125,1.618816,green,red
158,2023,2,0.12040,1.912990,green,red
159,2023,3,-0.87075,0.492591,green,red
160,2023,4,0.42620,0.875527,green,red
161,2023,5,-0.48200,1.689419,green,red


In [23]:
weekly_prices = (
    ba_detailed.groupby(['Year','Week_Number'], as_index=False)
    .agg(Open_w=('Open','first'), Close_w=('Close','last'))
    .sort_values(['Year','Week_Number'])
    .reset_index(drop=True)
)

weekly_prices_pred = weekly_prices.merge(test_df, on=['Year','Week_Number'], how='inner').sort_values(['Year','Week_Number']).reset_index(drop=True)
# weekly_prices_pred.head()


# Buy-and-Hold
weekly_prices_pred_yr = weekly_prices_pred.groupby(["Year"]).agg(
    Close_y = ('Close_w', 'last')
    ).reset_index().sort_values(['Year']).reset_index(drop=True)


shares = (100 / weekly_prices_pred.iloc[0]['Open_w'])
portfolio=weekly_prices_pred_yr.copy()
portfolio["BuyHold"] = round(shares * weekly_prices_pred_yr['Close_y'], 2)

print(portfolio)

   Year     Close_y  BuyHold
0  2023  260.660004   135.09
1  2024  176.550003    91.50


In [24]:
# Trading Strategy
cash = 100
shares = 0
results = {}

weekly_prices_pred = weekly_prices_pred.sort_values(['Year','Week_Number']).reset_index(drop=True)

for i in range(len(weekly_prices_pred)):    
    week = weekly_prices_pred.iloc[i]
    next_week = weekly_prices_pred.iloc[i+1] if i+1 < len(weekly_prices_pred) else None
    
    if(shares == 0 and week['predicted_label'] == "green"):
        shares = cash / week['Open_w']
        cash = 0
    
    if (shares > 0 and ((next_week is None) or next_week['predicted_label'] == "red")):
        cash = shares * week['Close_w']
        shares = 0
        
    if week['Week_Number'] == 52: #store yearly earnings/losses
        yr_total = shares*week['Close_w'] if shares > 0 else cash
        results[week['Year']] = (yr_total)

portfolio["Exp.Dist"] = round(portfolio["Year"].map(results), 2)
print(portfolio)

   Year     Close_y  BuyHold  Exp.Dist
0  2023  260.660004   135.09       100
1  2024  176.550003    91.50       100


Since the model never predicted "green", the model never entered the market, resulting with the initial $100 cash balance for both years.

Thus Exponential Naive Bayes is not suitable for our dataset because the features are not exponentially distributed.